In [ ]:
import os
from glob import glob
import rasterio as rio
import numpy as np
import geopandas as gpd
import pandas as pd
from sklearn.metrics import confusion_matrix
import geemap
import ee

ee.Initialize()
ee.Authenticate()

from LC_funcs import *


Successfully saved authorization token.


1) download Mapbiomas-Paraguay from gee or copy ready files for CEL-Pry classifications 
2) create mosaic of tiles
3) relcassify mosaic
4) extract raster vals at 1km x 1km sample points 
5) save agreement matrix 

In [ ]:
## MapBiomas Paraguay

reclass_csv = "/home/downspout-cel/paraguay_lc/lc_prods/recode/MB_PRY_recode.csv"

years=list(range(2018, 2023, 1))
prod_dir="/home/downspout-cel/paraguay_lc/lc_prods/MB_PRY"
grid_file="/home/downspout-cel/paraguay_lc/Segmentations/PY_grid_8858.gpkg"

######################################################################################
# ## 1) download MB -- error when downoading as one image so download each image tile 
MB_PRY_GEE(years, out_dir, grid_file)

# ## 2) mosaic MB 
mosaics=[]
for year in years:
    out_mos=os.path.join(prod_dir, "_".join([os.path.basename(prod_dir), str(year)+".tif"]))
    if not os.path.exists(out_mos):
        file_path_list = [os.path.join(prod_dir, "_".join([os.path.basename(prod_dir), str(year)]), fi) for fi in os.listdir(os.path.join(prod_dir, "_".join([os.path.basename(prod_dir), str(year)]))) if fi.endswith(".tif")] 
        mosaics.append(mosaic_rasters(file_path_list, out_mos))
    print(out_mos)
print(mosaics)

## 3) reclass MB
for mosaic in mosaics:
    print(mosaic)
    reclassify_raster(raster_path=mosaic, old_new=reclass_csv, out_dir=prod_dir)


In [163]:
## CEL

prod_dir="/home/downspout-cel/paraguay_lc/lc_prods/CEL"

classifications="/home/downspout-cel/paraguay_lc/stac/grids/"
filter_string="base1000"
year=str(2022)

######################################################################################
## 1) find grids cells that are ready, copy to lc_prod/CEL directory
for full_dir in [os.path.join(classifications, i, "comp") for i in sorted([ i for i in os.listdir(classifications) if len(i)==6])]:
    if os.path.exists(full_dir):
        class_files=[i for i in os.listdir(full_dir) if filter_string in i]
        if len(class_files) > 0:
            file_to_copy = os.path.join(full_dir, class_files[0])
            if not os.path.exists(os.path.join(prod_dir, file_to_copy)):
                !cp $file_to_copy $prod_dir
## 2) mosaic
rast_mosaic = os.path.join(prod_dir, "_".join([os.path.basename(prod_dir), str(year)+".tif"]))
mosaic_rasters(sorted([i for i in os.listdir(prod_dir) if (filter_string in i and i.endswith(".tif"))]), out_mos=rast_mosaic)
## 3) reclass
reclassify_raster(raster_path=rast_mosaic, 
                  old_new="/downspout-cel/paraguay_lc/lc_prods/recode/"+os.path.basename(prod_dir)+"_recode.csv", 
                  out_dir=prod_dir)

In [ ]:
## BAUMANN 

prod_dir = "/home/downspout-cel/paraguay_lc/lc_prods/Baumann"

######################################################################################
## 3) reclass 
for rast in [os.path.join(prod_dir, i) for i in os.listdir(prod_dir) if ("ForestLoss" not in i and "reclass" not in i)]:
    print(rast)
    reclassify_raster(raster_path=rast, 
                      old_new="/home/downspout-cel/paraguay_lc/lc_prods/recode/"+os.path.basename(prod_dir)+"_recode.csv", 
                      out_dir=prod_dir)

In [ ]:
## 4) extract raster vals at sample points 

import warnings
warnings.filterwarnings('ignore')

samp_pts="/home/downspout-cel/paraguay_lc/lc_prods/fishnet_1km.csv"
df = pd.read_csv(samp_pts)
coords_list = [np.float64(i[7:-1].split(" ")) for i in df['geometry']]

for rast in glob("/home/downspout-cel/paraguay_lc/lc_prods/**/*_reclass.tif", recursive=True):
    with rio.open(rast) as src:
        ## create column for raster, row is raster value at each coordinate
        field_pts[os.path.basename(rast).replace("_reclass.tif", "")] = [x[0] for x in src.sample([transform_point_coords(inepsg=CRS.from_user_input(8858), outepsg=src.crs, XYcoords=i) for i in coords_list])]
    print(rast)
## remove rows where they're ALL zero
df = field_pts.loc[field_pts['CEL'] * field_pts['Baumann'] * field_pts['MB_PRY'] != 0]
df.to_csv(samp_pts.replace(".csv", "_vals.csv"))

In [ ]:
## 5) save agreement matrix 

for LC_prods in [["CEL", "Baumann"], ["MB_PRY", "Baumann"], ["CEL", "MB_PRY"]]:

    ## subset cols 
    matrix_df = df[[LC_prods[0], LC_prods[1]]]
    ## delete if ANY are zero
    matrix_df = matrix_df[(matrix_df != 0).all(1)]

    ## first is true value, second
    true = matrix_df[LC_prods[0]]
    pred =  matrix_df[LC_prods[1]]
    agree_arr = confusion_matrix(true, pred)
    df_ag = pd.DataFrame(agree_arr)
    df_ag.to_csv("/home/downspout-cel/paraguay_lc/lc_prods/"+"_".join([os.path.basename(samp_pts)[:-4],"agree",LC_prods[0],LC_prods[1]+".csv"]))